In [ ]:
#!pip install pdfminer

In [1]:
pwd()

'C:\\Users\\sarathbabu.karunanit\\Downloads\\Resume'

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
import glob
files=glob.glob('C:\\Users\\sarathbabu.karunanit\\Downloads\\Resume\\Yet to Screen Profiles\\*.pdf')
print(len(files))

30


In [4]:
#import textract
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def resume_extraction(file):
    file_name=file.split('\\')
    file_name=file_name[-1]
    file_name

    text=convert_pdf_to_txt(file)
    len(text)

    number=re.findall(r'[0-9]+', text)
    ph_number=[i for i in number if len(i)==10]
    if len(ph_number)>0:
        ph_number=ph_number[0]
    else:
        ph_number=''
    ph_number
    
    #----------------Landline Numbers-----------------#
    if ph_number=='':
        ph_number=[i for i in number if len(i)==11]
        if len(ph_number)>0:
            ph_number=ph_number[0]
            if ph_number[:1]=='0':
                ph_number=ph_number[-10:]
        else:
            ph_number=''
    
    #---------------Numbers prefixed with +91--------#
    if ph_number=='':
        ph_number=[i for i in number if len(i)==12]
        if len(ph_number)>0:
            ph_number=ph_number[0]
            if ph_number[:2]=='91':
                ph_number=ph_number[-10:]
        else:
            ph_number=''
    ph_number

    if ph_number=='':
        for i in range(len(number)-1):
            if number[i][0]=='9' or number[i][0]=='8' or number[i][0]=='7' or number[i][0]=='6':
                if len(number[i])+len(number[i+1])==10:
                    ph_number=number[i]+number[i+1]
                    break
    ph_number

    #print(number)
    if ph_number=='':
        for i in range(len(number)-2):
            if number[i][0]=='9' or number[i][0]=='8' or number[i][0]=='7' or number[i][0]=='6':
                if len(number[i])+len(number[i+1])+len(number[i+2])==10:
                    ph_number=number[i]+number[i+1]+number[i+2]
                    break
    ph_number

    
    stopwords=pd.read_csv('stopwords.csv')
    stop_words=set(stopwords['stopwords'].values)

    # Convert all strings to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub('[^a-z0-9@.+]',' ',text)

    text=' '.join([i for i in text.split(' ') if i not in stop_words])

    def years_of_experience(string):
        exp=[]
        exp_index=text.find(string)
        for i in range(exp_index):
            if text[exp_index-i-1]==" ":
                 break
            exp.append(text[exp_index-i-1])
        exp=exp[::-1]
        exp=''.join(exp)
        return exp

    def experience_of_years(string):
        exp=[]
        exp_index=text.find(string)
        exp_index=exp_index+len(string)
        for i in range(exp_index,exp_index+25):
            if text[i]==" ":
                break
            exp.append(text[i])
        exp=''.join(exp)
        return exp

    #print(text.find('yrs'))
    text=re.sub('yrs','years',text)
    text=re.sub('experienced','',text)
    #text=re.sub('of','',text)

    yoe=['years',' years','  years','    years']
    eoy=['experience','experience ','experience  ','experience   ']

    if text.find('years')<text.find('experience'):
        #print('years')
        exp_list=list(filter(None,map(years_of_experience,yoe)))
    else:
        #print('experience')
        exp_list=list(filter(None,map(experience_of_years,eoy)))
    if len(exp_list)>0:
        exp=exp_list[0]
    else:
        exp=''
    
    exp=re.sub('[^0-9+.]','',exp)

    email=[]
    com=[]
    email_index=text.find('@')

    for i in range(email_index):
        if text[email_index-i-1]==" ":
             break
        email.append(text[email_index-i-1])

    for i in range(email_index,email_index+25):
        if text[i+1]==" ":
            break
        com.append(text[i+1])

    email=''.join(email)
    email=email[::-1]
    com=''.join(com)
    email_address=email+'@'+com
    email_address

    name=re.sub('[^a-z]','',email)
    name

    cols=['Resume','Name','Email','Phone','Years of Experience']
    values=[file_name,name,email_address,ph_number,exp]
    df_personal=pd.DataFrame([values],columns=cols)
    df_personal

    #df_skills=pd.read_csv('resume_skills.csv')
    df_skills=pd.read_excel('resume_skills.xlsx')
    df_skills=df_skills.where(df_skills.notnull(),None)
    df_skills=df_skills.transpose()

    skills_list=df_skills.values.tolist()
    skills_desc=df_skills.index

    skills_list=[list(filter(None,i)) for i in skills_list]

    def skills(*args):
        x=args[0]
        y=args[1]
        x=[i for i in x if i in text]
        df=pd.DataFrame(zip([x]),columns=[y])
        df[y]=df[y].apply(lambda z:','.join(z))
        return df
    df_skills=pd.concat(map(skills,skills_list,skills_desc),axis=1)
    df_skills

    df=pd.concat([df_personal,df_skills],axis=1)
    
    df['Shortlisted(Yes/No)']=''
    cols=list(df.columns.values)
    cols.remove('Resume')
    cols.append('Resume')
    
    df=df[cols]
    
    return df


In [6]:
len(files)

30

In [7]:
file=files[5]
file

'C:\\Users\\sarathbabu.karunanit\\Downloads\\Resume\\Yet to Screen Profiles\\f1503912531Dot9665.pdf'

In [8]:
from time import time

In [9]:
start=time()
df=pd.concat(map(resume_extraction,files))
df.to_excel('Resume_to_be_shortlisted.xlsx',index=False)
end=time()
end-start

54.804319620132446

In [10]:
df

,Name,Email,Phone,Years of Experience,Education,Positions held,Programming Languages,Machine Learning,Algorithms,Data Visualization,Analytics Companies,MNC,Industry experience,Cloud,Shortlisted(Yes/No),Resume
0,ajayrout,ajayrout633@gmail.com,7030530748,7+,"mba,b.com",analyst,"python,r,sql","data science,machine learning","regression,clustering",qlikview,,,"banking,finance,retail",,,Ajaya Ku Rout_DS[7].pdf
0,hiamangarg,hiamangarg@gmail.com,8814856198,5,,data scientist,"python,r,sql","statistics,machine learning,deep learning,neur...","regression,clustering",,,tata,insurance,,,AmanGargResume.pdf
0,ankvyas,ankvyas13@gmail.com,8750343556,,b.tech,analyst,"python,r,sql","statistics,nlp,keras","forecasting,regression,clustering",tableau,,hcl,banking,aws,,ANKITVYAS[2_0].pdf
0,devchayan,dev.chayan@gmail.com,8617419418,2+,"b.e,b.sc,m.sc","analyst,data analyst","python,r,sql","machine learning,deep learning,nlp,keras,tenso...",classification,powerbi,,,,,,CHAYANKANTIDEV[2_2].pdf
0,dibyakantabehera,dibyakantabehera@live.com,9438021169,5,,data scientist,"python,r,sql","data science,statistics,machine learning,keras...",ann,,,cognizant,,azure,,Dibya Resume.pdf
0,gpoojitha,gpoojitha40@gmail.com,9505845942,3,b.tech,"analyst,data analyst,data scientist,decision s...","python,r,sql",data science,"regression,clustering","tableau,power bi",musigma,,insurance,"aws,gcp",,f1503912531Dot9665.pdf
0,sgkarthik,sgkarthik10@gmail.com,8098068602,,b.e,analyst,"python,r,sql","data science,statistics,machine learning,deep ...","regression,classification,clustering,ann",,,accenture,retail,gcp,,karthik[3_2] (1).pdf
0,sgkarthik,sgkarthik10@gmail.com,8098068602,,b.e,analyst,"python,r,sql","data science,statistics,machine learning,deep ...","regression,classification,clustering,ann",,,accenture,retail,gcp,,KARTHIK_SDotG_Machine_Learning_Engineer1.pdf
0,kritikk,kriti92.kk@gmail.com,9021625207,3,,data scientist,"python,r,sql","data science,statistics,machine learning,deep ...","forecasting,clustering",tableau,,,energy,aws,,Kriti_CV_210521.pdf
0,kshitiz,kshitiz1896@outlook.com,9996095757,3,,"analyst,data scientist,decision scientist","python,r,sql",,"regression,classification,ann",power bi,mu sigma,,retail,,,kshitiz1896@outlookDotcom.pdf


In [ ]:
resume_extraction(file)